In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
import os

https://www.youtube.com/watch?v=J5_-l7WIO_w&list=PLKnIA16_RmvaTbihpo4MtzVm4XOQa0ER0&index=17

check the improvement part 

In [2]:
# Load environment variables from the .env file
load_dotenv()

# # Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Initialize the embeddings object
gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=gemini_api_key
)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=gemini_api_key, temperature=0.3)


#### Step 1a - Indexing (Document Ingestion)

In [8]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

video_id = "Gfr50f6ZBvo"  # Only the ID

try:
    # Fetch transcript in English
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])
    
    # Combine all segments into a single string
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"Error occurred: {e}")


Error occurred: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'


In [ ]:
transcript_list

#### Step 1b - Indexing (Text Splitting)

In [9]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

NameError: name 'transcript' is not defined

In [10]:
len(chunks)

NameError: name 'chunks' is not defined

In [ ]:
chunks[100]

#### Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [11]:
vector_store = Chroma(
    embedding=gemini_embeddings,
    documents = chunks
    persist_directory='my_chroma_db',
    collection_name='sample'
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3132341182.py, line 3)

In [ ]:
vector_store.index_to_docstore_id

In [ ]:
vector_store.get_by_ids(['2436bdb8-3f5f-49c6-8915-0c654c888700'])

#### Step 2 - Retrieval

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

NameError: name 'vector_store' is not defined

In [13]:
retriever

NameError: name 'retriever' is not defined

In [14]:
retriever.invoke('What is deepmind')

NameError: name 'retriever' is not defined

#### Step 3 - Augmentation

In [15]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [16]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

NameError: name 'retriever' is not defined

In [17]:
retrieved_docs

NameError: name 'retrieved_docs' is not defined

In [18]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

NameError: name 'retrieved_docs' is not defined

In [19]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

NameError: name 'context_text' is not defined

In [20]:
final_prompt

NameError: name 'final_prompt' is not defined

#### Step 4 - Generation

In [21]:
answer = gemini_model.invoke(final_prompt)
print(answer.content)

NameError: name 'final_prompt' is not defined

#### Building a Chain

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [23]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [24]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

NameError: name 'retriever' is not defined

In [25]:
parallel_chain.invoke('who is Demis')

NameError: name 'parallel_chain' is not defined

In [26]:
parser = StrOutputParser()

In [27]:
main_chain.invoke('Can you summarize the video')

NameError: name 'main_chain' is not defined